# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import numpy as np 
import pandas as pd 
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil
from azureml.core import Environment

# Check the core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace and Experiment

In [2]:
# initialized a workspace from a persisted configuration
ws = Workspace.from_config()

project_folder = './hyper_drive_proj'
# choose a name for experiment
experiment_name = 'hyperd-heart-failure-exp'

experiment = Experiment(ws, experiment_name)

# get workspace details and print
output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group 
output['Location'] = ws.location 
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
output_df = pd.DataFrame(data = output, index=[''])
output_df.T

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


,
Subscription ID,e36180a3-997b-4765-87a8-36f70eefc1fb
Workspace,ws-airevo
Resource Group,rg-ai
Location,westus2
Experiment Name,hyperd-heart-failure-exp


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

## Create a compute target

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = 'cpu-cluster'

try:
    # check for existing compute target
    training_cluster = ComputeTarget(
        workspace=ws,
        name=cluster_name,
    )
    print('Found existing cluster, use it.')
except:
    # create compute target if non exists
    try:
        compute_config = AmlCompute.provisioning_configuration(
            vm_size='STANDARD_DS12_V2',
            max_nodes=4
        )
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [11]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1, delay_evaluation=3)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    '--C': uniform(0.001, 1.0),
    '--max_iter': choice(0, 10, 50, 100, 150, 200)
})

if "outputs" not in os.listdir():
    os.mkdir("./outputs")

train_script = "./outputs"
# Create a SKLearn estimator for use with train.py
# Copy training script into train_model directory
shutil.copy('train.py', train_script)

#TODO: Create your estimator and hyperdrive config

from azureml.core import ScriptRunConfig, Environment

#sklearn_env = Environment.get(workspace=ws, name='AzureML-Tutorial')

src = ScriptRunConfig(
    source_directory=train_script,
    script='train.py',
    compute_target=training_cluster
    #environment=sklearn_env
    )

hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
    max_concurrent_runs=3
)

In [14]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:

# evaluate the the run is indeed complete
assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
# get the best run
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

In [ ]:
print(best_run.get_file_names())

In [ ]:
#TODO: Save the best model

In [ ]:
# download the best run and register the model
best_run.download_file(name='outputs/model.joblib', output_file_path='./outputs')
model = best_run.register_model(model_name='hyperdrive_run', model_path='outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service